# **MultiClass Image Classification** <br/>
#### *An overview of evaluation metrics for a multiclass machine-learning model*

Whether it’s spelled multi-class or multiclass, the science is the same. Multiclass image classification is a common task in computer vision, where we categorize an image into three or more classes.

We have heard about classification and regression techniques in Machine Learning. We know that these two techniques work on different algorithms for discrete and continuous data respectively. In this article, we will learn more about classification.



Work Flow:<br/>
- Scrape Images from Google
- Removing duplicated images
- Dominate background color
- Training a model
- Predicting an Image-category
- When we can classify an image into more than one class, it is known as a multi-label image classification problem.

### **Initial Start-ups:**


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import csv
import glob
import pickle
import time
from simple_image_download import simple_image_download
from sklearn.cluster import KMeans
from PIL import Image, ImageStat
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, load_img,   ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: ignored

Make a CSV file for the categories you want to download and store that in a project directory. we will be using 16 categories in this project.

### **Step 1: Fetch Images from Google**

In [ ]:
def images_scrapped_from_google():
    number_of_images_download = 100
    response = simple_image_download.simple_image_download

    with open(load_queries_from_file) as file:
        queries = list(x[0] for x in csv.reader(file))
       
    for query in queries:
        response().download(query,number_of_images_download)

simple_image_download is open-source python library, which comes handy in-terms of downloading images from internet and to generate datasets for model training.

### **Step 2: Removing Duplicate Images from folder**

In [ ]:
folders = glob.glob(os.path.join(os.getcwd(), 'simple_images/*'))

images_storing = []
duplicated_files = []

for folder in folders:
    for image in glob.glob(folder + '/*'):
        if not image in duplicated_files:
            img_org = Image.open(image)
            pixel_mean1 = ImageStat.Stat(img_org).mean

            for image_2 in glob.glob(folder + '/*'):
                if image != image_2:
                    img_check = Image.open(image_2)
                    pixel_mean2 = ImageStat.Stat(img_check).mean

                    if pixel_mean1 == pixel_mean2:
                        duplicated_files.append(image)
                        duplicated_files.append(image_2)

                        try:
                            os.remove(image_2)
                        except:
                            pass

                        if not image in images_storing:
                            images_storing.append(image)
                else:
                    if not image in images_storing:
                        images_storing.append(image)

return images_storing

SyntaxError: ignored

### **What is glob.glob?**

So, it will lets you grab all the files or folders from directory using Regex. For example,

In [ ]:
folders = glob.glob(“simple_images/*”)
print(folders)

### **Step 3: Finding Dominant Background color**

In [ ]:
def getting_percentage_of_dominant_colors(cluster, centroids):
    labels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
    (hist, _) = np.histogram(cluster.labels_, bins=labels)
    hist = hist.astype("float")
    hist /= hist.sum()

    # iterate through each cluster's color and percentage
    colors = sorted([(percent, color) for (percent, color) in zip(hist, centroids)])
    for (percent, color) in colors:
        try:
            if percent > 0.50:
                print(color, "{:0.2f}%".format(percent * 100))
                return True
        except Exception as e:
            print(str(e))
    return False

Function will check if an image has a dominant color background > 0.50 then image will be stored otherwise delete it. The reason behind is that when you have a multi color in the background, the object won’t be accurately detected by Model.

In [ ]:
# Load images and convert to a list of pixels
load_images = images_scrapped_from_google()

images_for_model_train = []
count = 0
for img in load_images:
    try:
        count += 1
        print(count)
        try:
            image = cv2.imread(img)
        except Exception:
            pass
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        reshape = image.reshape((image.shape[0] * image.shape[1], 3))

        # Find and display most dominant colors
        kmeans = KMeans(n_clusters=5).fit(reshape)
        visualize = getting_percentage_of_dominant_colors(kmeans, kmeans.cluster_centers_)
        if visualize is True:
            images_for_model_train.append(img)
            with open("labels_list_of_single_images.csv", "w", newline="") as f:
                columns = ['image_path', 'category']
                writer = csv.writer(f)
                writer.writerow(columns)
                for img_to_csv in images_for_model_train:
                    writer.writerow([img_to_csv, img_to_csv.split("\\")[-2]])
        else:
            try:
                os.remove(img)
            except:
                pass

    except:
        print("Unrecognized Input of an Image")
        os.remove(img)


Writing a CSV file with category, for giving path to model, from where it takes image and getting train on it.



### **Step 4: Load Model for Training**

Now we are all set to start coding for our model training. We will be using a pre-trained model InceptionV3 which has been trained on the image data having 1000 classes.

Why use a pre-trained CNN model? The initial layers of a CNN train on only low-level and mid-level features such as edges, lines, borders, etc. All kinds of images contain these features in them. These characteristics of a pre-trained CNN makes it very reusable. Hence it makes sense to use such pre-trained models that have been already trained on a large set of data, for which many companies have invested a lot of money.



In [ ]:
try:
    df = pd.read_csv(os.path.join(os.getcwd(), "labels_list_of_single_images.csv"))
    top_categories = sorted(list(df['category'].value_counts().index))
    target_labels = df['category']
    train_data = np.array(
        [img_to_array(load_img(train_img, target_size=(299, 299))) for train_img in df['image_path'].values.tolist()]) \
        .astype('float32')
    x_train, x_validation, y_train, y_validation = train_test_split(train_data, target_labels, test_size=0.2,
                                                                    random_state=100)

    print("x_train samples: ", x_train.shape)
    print("x_validation samples: ", x_validation.shape)

    y_train = pd.get_dummies(y_train.reset_index(drop=True)).values
    y_validation = pd.get_dummies(y_validation.reset_index(drop=True)).values

    # Train Generator
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=30,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip='true')

    train_generator = train_datagen.flow(x_train, y_train, shuffle=False, batch_size=40, seed=42)

    # Validation Generator
    val_datagen = ImageDataGenerator(rescale=1. / 255)

    val_generator = val_datagen.flow(x_validation, y_validation, shuffle=False, batch_size=40, seed=42)


# Model Intialize
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Add a fully-connected layer
    x = Dense(512, activation='relu')(x)
    predictions = Dense(16, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(Adam(lr=.0001), loss= 'categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    start = time.time()
    model.fit_generator(train_generator,
                        steps_per_epoch=len(x_train) // 40,
                        validation_data=val_generator,
                        validation_steps=len(x_validation) // 40,
                        epochs=5,
                        verbose=2)
    end = time.time()
    print("\nTotal Time Taken:", round((end - start) / 60, 2), "Minutes")

    try:
        file = open("multi_class_model.pkl", "wb")
        pickle.dump(model, file)
        print("Model Saved..!!")
       
    except Exception as e:
        print(str(e))

except Exception as e:
    print(str(e))

**Transfer Learning:**

To use a pre-trained model we need to keep all the previous layers as is and change only the final layer according to our use case. InceptionV3 has been trained on 1000 image classes. Our problem has only 16 different image classes. Hence, we will modify the last layer of InceptionV3 to 16 classes. Transfer Learning saves a lot of training time and development effort of the engineers.

ImageDataGenerator works for augmented the images. We can do a lot more preprocessing for data augmentations. Neural networks work better with a lot of data. Data augmentation is a strategy which we use at training time to increase the amount of data we have.

### **Step 5: Predicting a image category**

In [ ]:
# For Prediction
def predict_from_image(img_path):
    model_classes = pickle.load(open("multi_class_model.pkl", "rb"))
    print(model_classes.summary())
    img = image.load_img(img_path, target_size=(299, 299))
    img_tensor = image.img_to_array(img)  # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor,
                                axis=0)  # (1, height, width, channels),
                  #add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.

    pred = model_classes.predict(img_tensor)
    sorted_category_list = sorted(top_categories)
    predicted_class = sorted_category_list[np.argmax(pred)]

    return predicted_class, max(pred)


img_path = os.path.join(os.getcwd(), "baseball bat_3.jpg")
classes, prob = predict_from_image(img_path)
print(f"\n{classes}\n{prob[top_categories.index(classes)]}")

In [ ]:
baseball_bat
# Output :  0.9999959468841553

We have now successfully built a CNN model using Transfer Learning. With this approach, any Multi-class Image Classification problem can be tackled with good accuracy in a short span of time.

Happy Learning…!!!

Source : https://medium.com/geekculture/multiclass-image-classification-dcf9585f2ff9#:~:text=Multiclass%20image%20classification%20is%20a,discrete%20and%20continuous%20data%20respectively.